In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error

%matplotlib inline

Using TensorFlow backend.


In [2]:
dftrain=pd.read_csv('Train.csv')
dftest=pd.read_csv('Test.csv')

In [3]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18288 entries, 0 to 18287
Data columns (total 3 columns):
ID          18288 non-null int64
Datetime    18288 non-null object
Count       18288 non-null int64
dtypes: int64(2), object(1)
memory usage: 428.8+ KB


In [4]:
dftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5112 entries, 0 to 5111
Data columns (total 2 columns):
ID          5112 non-null int64
Datetime    5112 non-null object
dtypes: int64(1), object(1)
memory usage: 80.0+ KB


In [5]:
dftrain.head()

,ID,Datetime,Count
0,0,25-08-2012 00:00,8
1,1,25-08-2012 01:00,2
2,2,25-08-2012 02:00,6
3,3,25-08-2012 03:00,2
4,4,25-08-2012 04:00,2


In [6]:
dftrain.index=dftrain.Datetime

In [7]:
new_data=[]
print(len(dftrain.Count))
for i in range(0,len(dftrain.Count)):
  if i > len(dftrain.Count)-31:
    new_data.append(dftrain.Count[i])
  else:
    new_data.append(dftrain.Count[i+30])
target=[float(i) for i in new_data]

18288


In [8]:
data=dftrain.drop('Datetime',axis=1)



In [9]:
target=dftrain['Count']

In [10]:
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [11]:
len(model_fit.y)


14630

In [12]:
cols=data.columns
cols
len(prediction)

3658

In [ ]:
#converting predictions to dataframe
cols = data.columns
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,1):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', np.sqrt(mean_squared_error(pred[i], valid[i])))

In [14]:
prediction

array([[14630.        ,   206.01916   ],
       [14631.        ,   206.03799746],
       [14632.        ,   206.05654954],
       ...,
       [18284.99999996,   265.83559315],
       [18285.99999996,   265.85195291],
       [18286.99999996,   265.86831267]])

In [15]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(data)
target = target.reshape(-1,1)
target.shape
target = sc.fit_transform(target)

AttributeError: 'Series' object has no attribute 'reshape'

In [42]:
data=np.array(training_set_scaled)